# Static Condensation

Static condensation refers to the process of eliminating unknowns that are internal to elements from the global linear system. They are useful in standard methods and critical in methods like HDG.

NGSolve automates static condensation across a variety of methods via a classification of degrees of freedom.

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh, order=8, dirichlet='bottom|right')
u = fes.TestFunction()
v = fes.TrialFunction()

### Telling BilinearForm to condense

In [2]:
a = BilinearForm(fes, flags={"eliminate_internal":True})
a += SymbolicBFI (grad(u) * grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI (1 * v)
f.Assemble()

u = GridFunction(fes)

* The assembled matrix $A=$ `a.mat` can be block partitioned into
$$
A =
\left(
\begin{array}{cc}
A_{LL} & A_{LE}
\\
A_{EL} & A_{EE}
\end{array}
\right)
$$
where
$L$ denotes the set of **local* or *internal**  degrees of freedom and $E$ denotes the set of **interface** degrees of freedom.

* In our current example $E$ consists of edge and vertex dofs while $L$ consists of triangle dofs.

* The condensed system is known as the **Schur complement** 
$$
S = A_{EL} - A_{EL} A_{LL}^{-1} A_{LE}.
$$

* When `eliminate_internal` is set to `True` in `a`, the statement `a.Assemble` actually assembles $S$.




### A factorization


* NGSolve provides 

    - `a.harmonic_extension_trans` 
$=
\left(\begin{array}{cc}
0 & 0 \\
-A_{EI} A_{LL}^{-1} & 0 
\end{array}\right)
$

    - `a.harmonic_extension` 
$=
\left(\begin{array}{cc}
0 & -A_{LL}^{-1} A_{LE}
\\
0 & 0
\end{array}\right)
$

    - `a.inner_solve`
    $=\left(\begin{array}{cc}
A_{LL}^{-1} & 0 
\\
0 & 0
\end{array}\right)$.

* To solve 
$$
\left(\begin{array}{cc}
A_{LL} & A_{LE}
\\
A_{EL} & A_{EE}
\end{array}\right)
\left(\begin{array}{c}
u_L \\ u_E
\end{array}\right)
= 
\left(\begin{array}{c}
f_L \\ f_E
\end{array}\right)
$$
we use the identity 
$$
\left(\begin{array}{c}
u_L \\ u_E
\end{array}\right)
=
\left(\begin{array}{cc}
I & -A_{LL}^{-1} A_{LE}
\\
0 & I 
\end{array}\right)
\left(\begin{array}{cc}
A_{LL}^{-1} & 0
\\
0 & S^{-1}
\end{array}\right)
\underbrace{
\left(\begin{array}{cc}
I & 0 \\
-A_{EL} A_{LL}^{-1} & I 
\end{array}\right)
\left(\begin{array}{c}
f_L \\ f_E
\end{array}\right)}_{
\left(\begin{array}{c}
f'_L \\ f'_E
\end{array}\right)
}
$$

### Steps to compute the solution

We implement the above formula step by step:

* The following step implements
$$
\left(\begin{array}{c}
f'_L \\ f'_E
\end{array}\right)
=
\left(\begin{array}{cc}
I & 0 \\
-A_{EL} A_{LL}^{-1} & I 
\end{array}\right)
\left(\begin{array}{c}
f_L \\ f_E
\end{array}\right).
$$

In [3]:
f.vec.data += a.harmonic_extension_trans * f.vec

* The next step implements
$$
\left(\begin{array}{c}
0 \\ u_E
\end{array}\right) = 
\left(\begin{array}{cc}
0 & 0 \\
0 & S^{-1}
\end{array}\right)
\left(\begin{array}{c}
f'_L \\ f'_E
\end{array}\right).
$$

In [4]:
u.vec.data = a.mat.Inverse(fes.FreeDofs(True)) * f.vec

* Next:
$$
\left(\begin{array}{c}
u'_L \\ u_E
\end{array}\right) = 
\left(\begin{array}{c}
0 \\ u_E
\end{array}\right)
+ 
\left(\begin{array}{cc}
A_{LL}^{-1} & 0 \\
0 & 0
\end{array}\right)
\left(\begin{array}{c}
f'_L \\ f'_E
\end{array}\right).
$$

In [5]:
u.vec.data += a.inner_solve * f.vec

* Finally:
$$
\left(\begin{array}{c}
u_L \\ u_E
\end{array}\right)
=
\left(\begin{array}{cc}
I & -A_{LL}^{-1} A_{LE}
\\
0 & I 
\end{array}\right)
\left(\begin{array}{c}
u_L' \\ u_E
\end{array}\right)
$$

In [6]:
u.vec.data += a.harmonic_extension * u.vec
Draw (u)

### Behind the scenes

How does NGSolve know what is in the index sets $L$ and $E$? 

* Look at `fes.CouplingType` to see a classification of degrees of freedom.


In [7]:
for i in range(fes.ndof):
    print(fes.CouplingType(i))

COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.WIREBASKET_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF
COUPLING_TYPE.INTERFACE_DOF


In [8]:
dof_types = {}
for i in range(fes.ndof):
    type = fes.CouplingType(i)
    if type in dof_types.keys():
        dof_types[type] += 1
    else:
        dof_types[type] = 1
dof_types

{COUPLING_TYPE.WIREBASKET_DOF: 20,
 COUPLING_TYPE.INTERFACE_DOF: 287,
 COUPLING_TYPE.LOCAL_DOF: 462}

The `LOCAL_DOF` forms the set $L$ and the remainder forms the set $E$. All finite element spaces in NGSolve have such dof classification. Through this mechanism a bilinear form is able to automatically compute the Schur complement and accompanying extension operators. Users need only specify the flag `eliminate_internal`. (Of course a users should also make sure their method has an invertible $A_{LL}$!)
